In [1]:
import pandas as pd
import numpy as np

import networkx as nx

In [2]:
countries = [
    {'id': '660', 'text': 'Anguilla'}, {'id': '28', 'text': 'Antigua and Barbuda'}, {'id': '32', 'text': 'Argentina'}, {'id': '533', 'text': 'Aruba'},
    {'id': '44', 'text': 'Bahamas'}, {'id': '52', 'text': 'Barbados'}, {'id': '84', 'text': 'Belize'}, {'id': '60', 'text': 'Bermuda'},
    {'id': '68', 'text': 'Bolivia (Plurinational State of)'}, {'id': '535', 'text': 'Bonaire'}, {'id': '76', 'text': 'Brazil'},
    {'id': '92', 'text': 'Br. Virgin Isds'}, {'id': '124', 'text': 'Canada'}, {'id': '136', 'text': 'Cayman Isds'}, {'id': '152', 'text': 'Chile'},
    {'id': '170', 'text': 'Colombia'}, {'id': '188', 'text': 'Costa Rica'}, {'id': '192', 'text': 'Cuba'},
    {'id': '531', 'text': 'Curaçao'}, {'id': '212', 'text': 'Dominica'}, {'id': '214', 'text': 'Dominican Rep.'}, {'id': '218', 'text': 'Ecuador'},
    {'id': '222', 'text': 'El Salvador'}, {'id': '238', 'text': 'Falkland Isds (Malvinas)'}, {'id': '254', 'text': 'French Guiana'}, {'id': '304', 'text': 'Greenland'},
    {'id': '308', 'text': 'Grenada'}, {'id': '312', 'text': 'Guadeloupe'}, {'id': '320', 'text': 'Guatemala'}, {'id': '328', 'text': 'Guyana'},
    {'id': '332', 'text': 'Haiti'}, {'id': '340', 'text': 'Honduras'}, {'id': '388', 'text': 'Jamaica'}, {'id': '474', 'text': 'Martinique'},
    {'id': '484', 'text': 'Mexico'}, {'id': '500', 'text': 'Montserrat'}, {'id': '558', 'text': 'Nicaragua'}, {'id': '591', 'text': 'Panama'},
    {'id': '600', 'text': 'Paraguay'}, {'id': '604', 'text': 'Peru'}, {'id': '461', 'text': 'Sabah'}, {'id': '652', 'text': 'Saint Barthelemy'},
    {'id': '659', 'text': 'Saint Kitts and Nevis'}, {'id': '658', 'text': 'Saint Kitts, Nevis and Anguilla'}, {'id': '662', 'text': 'Saint Lucia'},
    {'id': '534', 'text': 'Saint Maarten'}, {'id': '666', 'text': 'Saint Pierre and Miquelon'}, {'id': '670', 'text': 'Saint Vincent and the Grenadines'},
    {'id': '740', 'text': 'Suriname'}, {'id': '780', 'text': 'Trinidad and Tobago'}, {'id': '796', 'text': 'Turks and Caicos Isds'},
    {'id': '858', 'text': 'Uruguay'}, {'id': '850', 'text': 'US Virgin Isds'}, {'id': '842', 'text': 'USA'}, {'id': '841', 'text': 'USA (before 1981)'},
    {'id': '862', 'text': 'Venezuela'},
]

In [3]:
amer_id = set(int(c['id']) for c in countries)

### OIL DATA

In [93]:
oil_df = pd.read_csv('data/americas-oil-trade-2011-2021.csv', dtype={
    'Year': 'int', 'Reporter Code': 'int', 'Partner Code': 'int', 'Commodity Code': 'int'
})

In [94]:
oil_df.head()

,Year,Trade Flow,Reporter Code,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit,Qty,Trade Value (US$)
0,2011,Import,28,Antigua and Barbuda,ATG,0,World,WLD,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.500000e+01,5.230000e+02
1,2011,Import,28,Antigua and Barbuda,ATG,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.500000e+01,5.230000e+02
2,2011,Import,32,Argentina,ARG,0,World,WLD,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,8.040000e+02,5.050000e+03
3,2011,Export,32,Argentina,ARG,0,World,WLD,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,3.023702e+09,2.182224e+09
4,2011,Import,32,Argentina,ARG,24,Angola,AGO,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,3.500000e+01,5.950000e+02


In [95]:
country_codes = pd.concat([
    oil_df[['Reporter Code', 'Reporter']].rename(columns={'Reporter Code':'Code', 'Reporter':'Name'}),
    oil_df[['Partner Code', 'Partner']].rename(columns={'Partner Code':'Code', 'Partner':'Name'}),
]).groupby('Code').agg({
    'Name': 'last'
}).reset_index()

In [96]:
country_codes.to_dict(orient='records')

[{'Code': 0, 'Name': 'World'},
 {'Code': 8, 'Name': 'Albania'},
 {'Code': 12, 'Name': 'Algeria'},
 {'Code': 24, 'Name': 'Angola'},
 {'Code': 28, 'Name': 'Antigua and Barbuda'},
 {'Code': 31, 'Name': 'Azerbaijan'},
 {'Code': 32, 'Name': 'Argentina'},
 {'Code': 36, 'Name': 'Australia'},
 {'Code': 40, 'Name': 'Austria'},
 {'Code': 44, 'Name': 'Bahamas'},
 {'Code': 52, 'Name': 'Barbados'},
 {'Code': 56, 'Name': 'Belgium'},
 {'Code': 60, 'Name': 'Bermuda'},
 {'Code': 68, 'Name': 'Bolivia (Plurinational State of)'},
 {'Code': 76, 'Name': 'Brazil'},
 {'Code': 84, 'Name': 'Belize'},
 {'Code': 86, 'Name': 'Br. Indian Ocean Terr.'},
 {'Code': 92, 'Name': 'Br. Virgin Isds'},
 {'Code': 96, 'Name': 'Brunei Darussalam'},
 {'Code': 100, 'Name': 'Bulgaria'},
 {'Code': 112, 'Name': 'Belarus'},
 {'Code': 120, 'Name': 'Cameroon'},
 {'Code': 124, 'Name': 'Canada'},
 {'Code': 136, 'Name': 'Cayman Isds'},
 {'Code': 148, 'Name': 'Chad'},
 {'Code': 152, 'Name': 'Chile'},
 {'Code': 156, 'Name': 'China'},
 {'Co

#### EXCLUDE AGGREGATES

In [97]:
exclude_codes = [899, 838, 837, 637, 568, 490, 473, 0]

oil_df = oil_df.loc[
    ~oil_df['Partner Code'].isin(exclude_codes)
]

### ADD EDGES/CONNECTIONS

In [169]:
def add_edge(row):
    #edge = frozenset([row['Year'], row['Partner ISO'], row['Reporter ISO']])
    #edge = frozenset([row['Partner ISO'], row['Reporter ISO']])
    edge = frozenset([
        row['Partner ISO'],
        row['Reporter ISO'],
        #row['Trade Flow']
    ])
    return edge

oil_df.loc[:, 'Edge'] = oil_df.apply(add_edge, axis=1)

In [170]:
oil_df.head()

,Year,Trade Flow,Reporter Code,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit,Qty,Trade Value (US$),Edge
1,2011,Import,28,Antigua and Barbuda,ATG,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.500000e+01,523.0,"(ATG, USA)"
4,2011,Import,32,Argentina,ARG,24,Angola,AGO,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,3.500000e+01,595.0,"(ARG, AGO)"
5,2011,Import,32,Argentina,ARG,76,Brazil,BRA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.600000e+01,1266.0,"(BRA, ARG)"
6,2011,Export,32,Argentina,ARG,76,Brazil,BRA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,2.090765e+08,169565732.0,"(BRA, ARG)"
7,2011,Export,32,Argentina,ARG,152,Chile,CHL,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.029503e+09,717117739.0,"(ARG, CHL)"


In [171]:
def edge(*args):
    return frozenset(args)

In [172]:
oil_df.loc[
    oil_df['Edge'] == edge('ARG', 'USA')
]

,Year,Trade Flow,Reporter Code,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit,Qty,Trade Value (US$),Edge
11,2011,Import,32,Argentina,ARG,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,6.890000e+02,2.558000e+03,"(ARG, USA)"
12,2011,Export,32,Argentina,ARG,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.196550e+09,8.729731e+08,"(ARG, USA)"
290,2011,Import,842,USA,USA,32,Argentina,ARG,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.460553e+09,1.116314e+09,"(ARG, USA)"
359,2012,Import,32,Argentina,ARG,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.378000e+03,4.712000e+03,"(ARG, USA)"
360,2012,Export,32,Argentina,ARG,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.058099e+09,8.010938e+08,"(ARG, USA)"
647,2012,Import,842,USA,USA,32,Argentina,ARG,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.121349e+09,9.461250e+08,"(ARG, USA)"
700,2013,Import,32,Argentina,ARG,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,8.600000e+01,1.157000e+03,"(ARG, USA)"
701,2013,Export,32,Argentina,ARG,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,6.525120e+08,4.656301e+08,"(ARG, USA)"
714,2014,Import,32,Argentina,ARG,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.000000e+00,5.200000e+01,"(ARG, USA)"
715,2014,Export,32,Argentina,ARG,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.490210e+09,9.923607e+08,"(ARG, USA)"


### SIDE BY SIDE CONNECTIONS

In [201]:
show_cols = ['Edge', 'Year', 'Trade Flow_i', 'Reporter_i', 'Partner_i', 'Trade Value (US$)_i', 'Trade Flow_e', 'Reporter_e', 'Partner_e', 'Trade Value (US$)_e']

In [202]:
oil_merged = oil_df.merge(
    oil_df,
    on = ['Year', 'Edge'],
    how = 'left',
    suffixes = ('_i', '_e')
)

In [203]:
oil_merged = oil_merged.loc[
    oil_merged['Reporter_i'] != oil_merged['Reporter_e'],
    show_cols
]

In [204]:
oil_merged.loc[
    oil_merged['Trade Flow_i'] == oil_merged['Trade Flow_e'],
    ['Trade Flow_e', 'Reporter_e', 'Partner_e', 'Trade Value (US$)_e']
] = np.nan

In [205]:
oil_merged.query('Year == 2011').head(20)

,Edge,Year,Trade Flow_i,Reporter_i,Partner_i,Trade Value (US$)_i,Trade Flow_e,Reporter_e,Partner_e,Trade Value (US$)_e
7,"(ARG, CHL)",2011,Export,Argentina,Chile,7.171177e+08,Import,Chile,Argentina,5.987843e+08
13,"(ARG, USA)",2011,Import,Argentina,USA,2.558000e+03,NaN,NaN,NaN,NaN
16,"(ARG, USA)",2011,Export,Argentina,USA,8.729731e+08,Import,USA,Argentina,1.116314e+09
20,"(TTO, BRB)",2011,Import,Barbados,Trinidad and Tobago,1.712266e+07,NaN,NaN,NaN,NaN
23,"(TTO, BRB)",2011,Export,Barbados,Trinidad and Tobago,1.961851e+07,Import,Trinidad and Tobago,Barbados,3.277556e+07
25,"(BLZ, USA)",2011,Export,Belize,USA,1.460437e+08,Import,USA,Belize,1.277511e+08
28,"(BRA, BOL)",2011,Import,Bolivia (Plurinational State of),Brazil,5.450000e+02,Export,Brazil,Bolivia (Plurinational State of),5.220000e+02
30,"(BOL, CHL)",2011,Export,Bolivia (Plurinational State of),Chile,5.860967e+07,Import,Chile,Bolivia (Plurinational State of),3.418101e+07
34,"(BOL, USA)",2011,Import,Bolivia (Plurinational State of),USA,1.057400e+04,NaN,NaN,NaN,NaN
37,"(BOL, USA)",2011,Export,Bolivia (Plurinational State of),USA,1.430260e+08,Import,USA,Bolivia (Plurinational State of),2.223474e+08


In [206]:
oil_merged.loc[
    oil_merged.Partner_e.isnull()
]

,Edge,Year,Trade Flow_i,Reporter_i,Partner_i,Trade Value (US$)_i,Trade Flow_e,Reporter_e,Partner_e,Trade Value (US$)_e
13,"(ARG, USA)",2011,Import,Argentina,USA,2.558000e+03,NaN,NaN,NaN,NaN
20,"(TTO, BRB)",2011,Import,Barbados,Trinidad and Tobago,1.712266e+07,NaN,NaN,NaN,NaN
34,"(BOL, USA)",2011,Import,Bolivia (Plurinational State of),USA,1.057400e+04,NaN,NaN,NaN,NaN
46,"(BRA, CAN)",2011,Import,Brazil,Canada,4.255000e+03,NaN,NaN,NaN,NaN
72,"(PER, BRA)",2011,Import,Brazil,Peru,1.560021e+08,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5455,"(USA, COL)",2021,Export,USA,Colombia,4.102389e+08,NaN,NaN,NaN,NaN
5461,"(USA, DOM)",2021,Export,USA,Dominican Rep.,4.085638e+08,NaN,NaN,NaN,NaN
5474,"(GTM, USA)",2021,Import,USA,Guatemala,5.161470e+07,NaN,NaN,NaN,NaN
5477,"(GUY, USA)",2021,Import,USA,Guyana,2.119151e+09,NaN,NaN,NaN,NaN


In [208]:
oil_df.loc[
    oil_df['Edge'] == edge('USA', 'DOM')
]

,Year,Trade Flow,Reporter Code,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit,Qty,Trade Value (US$),Edge
165,2011,Import,214,Dominican Rep.,DOM,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,4.830900e+04,27825.0,"(USA, DOM)"
166,2011,Export,214,Dominican Rep.,DOM,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.399800e+05,34963.0,"(USA, DOM)"
519,2012,Import,214,Dominican Rep.,DOM,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,7.231105e+07,60477508.0,"(USA, DOM)"
520,2012,Export,214,Dominican Rep.,DOM,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,6.741010e+05,26571.0,"(USA, DOM)"
1530,2013,Import,214,Dominican Rep.,DOM,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.948640e+05,162975.0,"(USA, DOM)"
1551,2014,Import,214,Dominican Rep.,DOM,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,2.280100e+04,96617.0,"(USA, DOM)"
1552,2014,Export,214,Dominican Rep.,DOM,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.901710e+05,7861.0,"(USA, DOM)"
1580,2015,Import,214,Dominican Rep.,DOM,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,3.480800e+04,144074.0,"(USA, DOM)"
1581,2015,Export,214,Dominican Rep.,DOM,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,4.354600e+04,1200.0,"(USA, DOM)"
1605,2016,Import,214,Dominican Rep.,DOM,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,3.935874e+07,14945578.0,"(USA, DOM)"


### NETWORKX

In [29]:
nodes = [
    (int(c.Code), {'name': c.Name}) for c in country_codes.itertuples()
]

edges

G = nx.DiGraph()

G.add_nodes_from(nodes)

In [31]:
#G.nodes.data()

In [26]:
oil_df.loc[
    oil_df['Reporter ISO'].isin(['USA', 'VEN']) & oil_df['Partner ISO'].isin(['USA', 'VEN'])
]

,Year,Trade Flow,Reporter Code,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit,Qty,Trade Value (US$)
340,2011,Import,842,USA,USA,862,Venezuela,VEN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,5.272747e+10,3.791359e+10
349,2011,Import,862,Venezuela,VEN,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.400000e+01,6.380000e+02
686,2012,Import,842,USA,USA,862,Venezuela,VEN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,4.654537e+10,3.464306e+10
693,2012,Import,862,Venezuela,VEN,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,6.000000e+00,3.070000e+02
2162,2013,Import,842,USA,USA,862,Venezuela,VEN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,3.857753e+10,2.815425e+10
2227,2014,Import,842,USA,USA,862,Venezuela,VEN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,3.223855e+10,2.641598e+10
2286,2015,Import,842,USA,USA,862,Venezuela,VEN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,3.957572e+10,1.374022e+10
2364,2016,Import,842,USA,USA,862,Venezuela,VEN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,4.383170e+10,9.767841e+09
2467,2017,Import,842,USA,USA,862,Venezuela,VEN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,2.670757e+10,1.071999e+10
3517,2018,Import,842,USA,USA,862,Venezuela,VEN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,2.018201e+10,1.062935e+10


In [27]:
oil_df.loc[
    oil_df['Reporter ISO'].isin(['VEN'])
]

,Year,Trade Flow,Reporter Code,Reporter,Reporter ISO,Partner Code,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit,Qty,Trade Value (US$)
342,2011,Import,862,Venezuela,VEN,0,World,WLD,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,7.700000e+01,3.061000e+03
343,2011,Export,862,Venezuela,VEN,0,World,WLD,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,7.751039e+10,6.091316e+10
344,2011,Export,862,Venezuela,VEN,473,"LAIA, nes",NaN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,3.613491e+10,2.839738e+10
345,2011,Export,862,Venezuela,VEN,490,"Other Asia, nes",NaN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.612096e+10,1.266900e+10
346,2011,Export,862,Venezuela,VEN,568,"Other Europe, nes",NaN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,4.342179e+09,3.412392e+09
347,2011,Import,862,Venezuela,VEN,591,Panama,PAN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,6.300000e+01,2.423000e+03
348,2011,Export,862,Venezuela,VEN,637,"North America and Central America, nes",NaN,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,2.091233e+10,1.643440e+10
349,2011,Import,862,Venezuela,VEN,842,USA,USA,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,1.400000e+01,6.380000e+02
687,2012,Import,862,Venezuela,VEN,0,World,WLD,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,6.000000e+00,3.070000e+02
688,2012,Export,862,Venezuela,VEN,0,World,WLD,270900,"Petroleum oils, oils from bituminous minerals,...",Weight in kilograms,8.614483e+10,6.891246e+10


In [7]:
np.union1d(oil_df['Reporter'], oil_df['Partner'])

array(['Albania', 'Algeria', 'Angola', 'Anguilla', 'Antigua and Barbuda',
       'Areas, nes', 'Argentina', 'Aruba', 'Australia', 'Austria',
       'Azerbaijan', 'Bahamas', 'Barbados', 'Belarus', 'Belgium',
       'Belize', 'Bermuda', 'Bolivia (Plurinational State of)', 'Bonaire',
       'Br. Indian Ocean Terr.', 'Br. Virgin Isds', 'Brazil',
       'Brunei Darussalam', 'Bulgaria', 'Bunkers', 'Cameroon', 'Canada',
       'Cayman Isds', 'Chad', 'Chile', 'China', 'China, Hong Kong SAR',
       'Colombia', 'Congo', 'Costa Rica', 'Croatia', 'Cuba', 'Curaçao',
       'Czechia', "Côte d'Ivoire", 'Dem. Rep. of the Congo', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Rep.', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Estonia', 'Finland', 'France',
       'Free Zones', 'Gabon', 'Georgia', 'Germany', 'Ghana', 'Gibraltar',
       'Greece', 'Greenland', 'Grenada', 'Guatemala', 'Guyana', 'Haiti',
       'Honduras', 'Hungary', 'India', 'Indonesia', 'Iran', 'Iraq',
       '

In [28]:
amer_oil_df = oil_df.loc[
    oil_df['Reporter Code'].isin(amer_id) & oil_df['Partner Code'].isin(amer_id)
]

In [29]:
np.union1d(amer_oil_df['Reporter'], amer_oil_df['Partner'])

array(['Anguilla', 'Antigua and Barbuda', 'Argentina', 'Aruba', 'Bahamas',
       'Barbados', 'Belize', 'Bermuda',
       'Bolivia (Plurinational State of)', 'Bonaire', 'Br. Virgin Isds',
       'Brazil', 'Canada', 'Cayman Isds', 'Chile', 'Colombia',
       'Costa Rica', 'Cuba', 'Curaçao', 'Dominica', 'Dominican Rep.',
       'Ecuador', 'El Salvador', 'Greenland', 'Grenada', 'Guatemala',
       'Guyana', 'Haiti', 'Honduras', 'Jamaica', 'Mexico', 'Montserrat',
       'Nicaragua', 'Panama', 'Paraguay', 'Peru', 'Saint Kitts and Nevis',
       'Saint Lucia', 'Saint Pierre and Miquelon',
       'Saint Vincent and the Grenadines', 'Suriname',
       'Trinidad and Tobago', 'Turks and Caicos Isds', 'USA', 'Uruguay',
       'Venezuela'], dtype=object)

### IMPORT/EXPORT TOTAL

In [32]:
amer_oil_df.loc[
    (amer_oil_df.Year == 2021) & (amer_oil_df['Trade Flow'] == 'Export')
].groupby(['Year', 'Reporter', 'Trade Flow', ]).agg({
    'Trade Value (US$)': 'sum'
}).reset_index().sort_values(by='Trade Value (US$)', ascending=False)

,Year,Reporter,Trade Flow,Trade Value (US$)
6,2021,Canada,Export,7.996191e+10
14,2021,USA,Export,1.163179e+10
5,2021,Brazil,Export,6.260061e+09
9,2021,Ecuador,Export,5.989016e+09
7,2021,Colombia,Export,5.959348e+09
11,2021,Guyana,Export,1.857407e+09
13,2021,Trinidad and Tobago,Export,1.153731e+09
0,2021,Argentina,Export,7.966386e+08
10,2021,Guatemala,Export,7.540806e+07
4,2021,Bolivia (Plurinational State of),Export,8.954099e+06


In [33]:
amer_oil_df.loc[
    (amer_oil_df.Year == 2021) & (amer_oil_df['Trade Flow'] == 'Import')
].groupby(['Year', 'Reporter', 'Trade Flow', ]).agg({
    'Trade Value (US$)': 'sum'
}).reset_index().sort_values(by='Trade Value (US$)', ascending=False)

,Year,Reporter,Trade Flow,Trade Value (US$)
12,2021,USA,Import,1.088459e+11
3,2021,Canada,Import,7.994134e+09
4,2021,Chile,Import,3.655821e+09
2,2021,Brazil,Import,2.487821e+09
6,2021,Dominican Rep.,Import,6.663544e+08
9,2021,Nicaragua,Import,3.844219e+08
5,2021,Colombia,Import,1.496574e+08
0,2021,Barbados,Import,1.974013e+06
7,2021,Guatemala,Import,3.556940e+05
8,2021,Guyana,Import,4.680000e+02
